In [697]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
import heapq

In [782]:
from sklearn.base import BaseEstimator
from statistics import mean
import statistics
import math

class KNearestClassifier(BaseEstimator):
    def __init__(self, k=1):
        self.k = k
    def fit(self, X, y):
        self.X = X
        self.y = y
    def predict(self, example):
        # case:general
        
        # calculate distances, stored into distance_dict
        distance_dict = {}
        for idx, x in enumerate(self.X):
            diff = (example-x)**2
            #distance_dict[diff] = idx 
            # if there are same diff, some distances will lose, since key is unique
            distance_dict[idx] = diff
        #print(len(distance_dict),'distance dict:',distance_dict)
        
        # use heap to get the k minimum distances, stored into k_min
        distances = []
        for idx, distance in distance_dict.items():
            heapq.heappush(distances, distance)
            
        k_min = []
        k = self.k
        #print(k, 'distances:', len(distances))
        while(k>0):
            k_min.append(heapq.heappop(distances))
            k -= 1
        #print(self.k, ' k_min:', k_min)
        
        # y of k nearest neighbors
        k_min_ret = []
        idxs = []
        for diff in k_min:
            for idx, value in distance_dict.items():
                if(value == diff):
                    if idx not in idxs:
                        idxs.append(idx)
                        k_min_ret.append(self.y.values[idx])
                        break
        
        print(self.k, ' k_min_ret:', k_min_ret, idxs)
        
        #majority
#         if(len(k_min_ret)==1):
#             return k_min_ret[0] 
        
    
        s = pd.Series(k_min_ret)
        value_counts = s.value_counts()
        max_count = 0
        max_value = None
        for value, count in value_counts.items():
            if(count > max_count):
                max_count = count
                max_value = value
                
        ret = {}
        ret[example] = max_value
        print(ret)
        return ret
    
    
        # case:k=1
#         min_idx = math.inf
#         min_value = math.inf
#         for idx, x in enumerate(self.X):
#             diff = (example-x)**2
#             if(diff < min_value):
#                 min_value = diff
#                 min_idx = idx
#         return self.y[min_idx]

In [783]:
def k_cross_validation(classifier, X, y, k=3):
    # folders
    folders_X = {}
    folders_y = {}
    n_elements = int(len(X)/k)
    #print('n_elements:', n_elements)
    for idx in range(k):
        folders_X[idx] = X[idx*n_elements:idx*n_elements+n_elements]
        folders_y[idx] = y[idx*n_elements:idx*n_elements+n_elements]
    #print(folders_X)
    #print(folders_y)
    
    errors = []
    for idx in range(k):
        X_test = folders_X[idx]
        y_test = folders_y[idx]
        
        X_train = pd.Series()
        y_train = pd.Series()
        for i in range(k):
            if(i != idx):
                X_train = pd.concat([X_train, folders_X[i]], axis=0)
                y_train = pd.concat([y_train, folders_y[i]], axis=0)
        
        #print('X_train:\n',len(X_train))
        #print('y_train:\n',len(y_train))
        X_test = X_test.values[0]
        #print('X_test:\n',X_test)
        
        classifier.fit(X_train, y_train)
        y_predict = classifier.predict(X_test)
        #print('prediction:', X_test, y_predict)
        
        y_predict_list  = list(y_predict.values())
        #print(y_predict_list)
        y_test_list = list(y_test)
        #print(y_test_list)
        
        diff = [test==pred for test, pred in zip(y_test_list, y_predict_list)]
        #print(diff)
        count = 0
        for element in diff:
            #print(element)
            if(element != True):
                count += 1
        errors.append(count)
    print('error count:', errors, len(errors))
    return mean(errors)

def cross_validation(classifier, X, y, k=3, cv = 1):
    errors = []
    for time in range(cv):
        e = k_cross_validation(classifier, X, y, k)
        errors.append(e)
    return mean(errors)
    

In [784]:
data = pd.DataFrame({
    'x': [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17],
    'y': ['A','A','A','A','B','A','A','A','A','B','B','B','B','A','B','B','B','B']})

X = data['x']
y = data['y']

In [785]:
knn_clf_1 = KNearestClassifier(1)
knn_clf_1.fit(X, y)
ret = knn_clf_1.predict(4.2)
print(ret)
ret = knn_clf_1.predict(0)
print(ret)
ret = knn_clf_1.predict(17)
print(ret)
ret = knn_clf_1.predict(18)
print(ret)

1  k_min_ret: ['B'] [4]
{4.2: 'B'}
{4.2: 'B'}
1  k_min_ret: ['A'] [0]
{0: 'A'}
{0: 'A'}
1  k_min_ret: ['B'] [17]
{17: 'B'}
{17: 'B'}
1  k_min_ret: ['B'] [17]
{18: 'B'}
{18: 'B'}


In [786]:
knn_clf_2 = KNearestClassifier(2)
knn_clf_2.fit(X, y)
ret = knn_clf_2.predict(4.2)
print(ret)

2  k_min_ret: ['B', 'A'] [4, 5]
{4.2: 'B'}
{4.2: 'B'}


In [787]:
knn_clf_3 = KNearestClassifier(3)
knn_clf_3.fit(X, y)
ret = knn_clf_3.predict(4.2)
print(ret)

3  k_min_ret: ['B', 'A', 'A'] [4, 5, 3]
{4.2: 'A'}
{4.2: 'A'}


In [788]:
k_folder = len(X)# left-one-out means only one instance per folder
print(k_folder)

18


In [789]:
knn_clf = KNearestClassifier()
e = cross_validation(knn_clf, X, y, k_folder, 1)
print(e)

1  k_min_ret: ['A'] [0]
{0: 'A'}
1  k_min_ret: ['A'] [0]
{1: 'A'}
1  k_min_ret: ['A'] [1]
{2: 'A'}
1  k_min_ret: ['A'] [2]
{3: 'A'}
1  k_min_ret: ['A'] [3]
{4: 'A'}
1  k_min_ret: ['B'] [4]
{5: 'B'}
1  k_min_ret: ['A'] [5]
{6: 'A'}
1  k_min_ret: ['A'] [6]
{7: 'A'}
1  k_min_ret: ['A'] [7]
{8: 'A'}
1  k_min_ret: ['A'] [8]
{9: 'A'}
1  k_min_ret: ['B'] [9]
{10: 'B'}
1  k_min_ret: ['B'] [10]
{11: 'B'}
1  k_min_ret: ['B'] [11]
{12: 'B'}
1  k_min_ret: ['B'] [12]
{13: 'B'}
1  k_min_ret: ['A'] [13]
{14: 'A'}
1  k_min_ret: ['B'] [14]
{15: 'B'}
1  k_min_ret: ['B'] [15]
{16: 'B'}
1  k_min_ret: ['B'] [16]
{17: 'B'}
error count: [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0] 18
0.2777777777777778


In [791]:
knn_clf = KNearestClassifier(2)
e = cross_validation(knn_clf, X, y, k_folder, 1)
print(e)

2  k_min_ret: ['A', 'A'] [0, 1]
{0: 'A'}
2  k_min_ret: ['A', 'A'] [0, 1]
{1: 'A'}
2  k_min_ret: ['A', 'A'] [1, 2]
{2: 'A'}
2  k_min_ret: ['A', 'B'] [2, 3]
{3: 'B'}
2  k_min_ret: ['A', 'A'] [3, 4]
{4: 'A'}
2  k_min_ret: ['B', 'A'] [4, 5]
{5: 'B'}
2  k_min_ret: ['A', 'A'] [5, 6]
{6: 'A'}
2  k_min_ret: ['A', 'A'] [6, 7]
{7: 'A'}
2  k_min_ret: ['A', 'B'] [7, 8]
{8: 'B'}
2  k_min_ret: ['A', 'B'] [8, 9]
{9: 'B'}
2  k_min_ret: ['B', 'B'] [9, 10]
{10: 'B'}
2  k_min_ret: ['B', 'B'] [10, 11]
{11: 'B'}
2  k_min_ret: ['B', 'A'] [11, 12]
{12: 'B'}
2  k_min_ret: ['B', 'B'] [12, 13]
{13: 'B'}
2  k_min_ret: ['A', 'B'] [13, 14]
{14: 'B'}
2  k_min_ret: ['B', 'B'] [14, 15]
{15: 'B'}
2  k_min_ret: ['B', 'B'] [15, 16]
{16: 'B'}
2  k_min_ret: ['B', 'B'] [16, 15]
{17: 'B'}
error count: [0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0] 18
0.2777777777777778


In [792]:
knn_clf = KNearestClassifier(3)
e = cross_validation(knn_clf, X, y, k_folder, 1)
print(e)

3  k_min_ret: ['A', 'A', 'A'] [0, 1, 2]
{0: 'A'}
3  k_min_ret: ['A', 'A', 'A'] [0, 1, 2]
{1: 'A'}
3  k_min_ret: ['A', 'A', 'A'] [1, 2, 0]
{2: 'A'}
3  k_min_ret: ['A', 'B', 'A'] [2, 3, 1]
{3: 'A'}
3  k_min_ret: ['A', 'A', 'A'] [3, 4, 2]
{4: 'A'}
3  k_min_ret: ['B', 'A', 'A'] [4, 5, 3]
{5: 'A'}
3  k_min_ret: ['A', 'A', 'B'] [5, 6, 4]
{6: 'A'}
3  k_min_ret: ['A', 'A', 'A'] [6, 7, 5]
{7: 'A'}
3  k_min_ret: ['A', 'B', 'A'] [7, 8, 6]
{8: 'A'}
3  k_min_ret: ['A', 'B', 'A'] [8, 9, 7]
{9: 'A'}
3  k_min_ret: ['B', 'B', 'A'] [9, 10, 8]
{10: 'B'}
3  k_min_ret: ['B', 'B', 'B'] [10, 11, 9]
{11: 'B'}
3  k_min_ret: ['B', 'A', 'B'] [11, 12, 10]
{12: 'B'}
3  k_min_ret: ['B', 'B', 'B'] [12, 13, 11]
{13: 'B'}
3  k_min_ret: ['A', 'B', 'B'] [13, 14, 12]
{14: 'B'}
3  k_min_ret: ['B', 'B', 'A'] [14, 15, 13]
{15: 'B'}
3  k_min_ret: ['B', 'B', 'B'] [15, 16, 14]
{16: 'B'}
3  k_min_ret: ['B', 'B', 'B'] [16, 15, 14]
{17: 'B'}
error count: [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0] 18
0.1666666666666666

In [793]:
knn_clf = KNearestClassifier(4)
e = cross_validation(knn_clf, X, y, k_folder, 1)
print(e)

4  k_min_ret: ['A', 'A', 'A', 'B'] [0, 1, 2, 3]
{0: 'A'}
4  k_min_ret: ['A', 'A', 'A', 'B'] [0, 1, 2, 3]
{1: 'A'}
4  k_min_ret: ['A', 'A', 'A', 'B'] [1, 2, 0, 3]
{2: 'A'}
4  k_min_ret: ['A', 'B', 'A', 'A'] [2, 3, 1, 4]
{3: 'A'}
4  k_min_ret: ['A', 'A', 'A', 'A'] [3, 4, 2, 5]
{4: 'A'}
4  k_min_ret: ['B', 'A', 'A', 'A'] [4, 5, 3, 6]
{5: 'A'}
4  k_min_ret: ['A', 'A', 'B', 'A'] [5, 6, 4, 7]
{6: 'A'}
4  k_min_ret: ['A', 'A', 'A', 'B'] [6, 7, 5, 8]
{7: 'A'}
4  k_min_ret: ['A', 'B', 'A', 'B'] [7, 8, 6, 9]
{8: 'B'}
4  k_min_ret: ['A', 'B', 'A', 'B'] [8, 9, 7, 10]
{9: 'B'}
4  k_min_ret: ['B', 'B', 'A', 'B'] [9, 10, 8, 11]
{10: 'B'}
4  k_min_ret: ['B', 'B', 'B', 'A'] [10, 11, 9, 12]
{11: 'B'}
4  k_min_ret: ['B', 'A', 'B', 'B'] [11, 12, 10, 13]
{12: 'B'}
4  k_min_ret: ['B', 'B', 'B', 'B'] [12, 13, 11, 14]
{13: 'B'}
4  k_min_ret: ['A', 'B', 'B', 'B'] [13, 14, 12, 15]
{14: 'B'}
4  k_min_ret: ['B', 'B', 'A', 'B'] [14, 15, 13, 16]
{15: 'B'}
4  k_min_ret: ['B', 'B', 'B', 'A'] [15, 16, 14, 13]
{16: 'B'

In [795]:
knn_clf = KNearestClassifier(17)
e = cross_validation(knn_clf, X, y, k_folder, 1)
print(e)

17  k_min_ret: ['A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B'] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
{0: 'B'}
17  k_min_ret: ['A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B'] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
{1: 'B'}
17  k_min_ret: ['A', 'A', 'A', 'B', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B'] [1, 2, 0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
{2: 'B'}
17  k_min_ret: ['A', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B'] [2, 3, 1, 4, 0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
{3: 'B'}
17  k_min_ret: ['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B'] [3, 4, 2, 5, 1, 6, 0, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
{4: 'A'}
17  k_min_ret: ['B', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B'] [4, 5, 3, 6, 2, 7, 1, 8, 0, 9, 10, 11, 12, 13, 14, 15,